<a href="https://colab.research.google.com/github/Jegovila/SI4/blob/main/5%20Clasificaci%C3%B3n%20n%C3%BAmeros%20y%20PCA/C%C3%B3digos/python/MNIST_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -nc "https://media.githubusercontent.com/media/Jegovila/SI4/main/5%20Clasificaci%C3%B3n%20n%C3%BAmeros%20y%20PCA/C%C3%B3digos/python/mnist_784.csv"
!wget -nc "https://github.com/Jegovila/SI4/raw/main/5%20Clasificaci%C3%B3n%20n%C3%BAmeros%20y%20PCA/C%C3%B3digos/python/MLP_MNIST.sav"
!wget -nc "https://raw.githubusercontent.com/Jegovila/SI4/main/5%20Clasificaci%C3%B3n%20n%C3%BAmeros%20y%20PCA/C%C3%B3digos/python/photo.jpg"

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import metrics

from sklearn.neural_network import MLPClassifier
import pickle

In [ ]:
data = pd.read_csv('mnist_784.csv')
print(data)

n = 70000 # datos
m = 28 # 28x28=784

x = np.asanyarray(data.drop(columns=['class']))
y = np.asanyarray(data[['class']])

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25)

¿Cuántos componentes?

In [ ]:
scaler = StandardScaler()
X = data.drop('class',axis=1)
X = scaler.fit_transform(X)

dfx = pd.DataFrame(data=X,columns=data.columns[1:])
pca = PCA(n_components=None)
pca.fit(dfx)
print(pca.explained_variance_)

plt.plot(pca.explained_variance_ratio_.cumsum(), marker = "o", linestyle="--")
plt.title("Explained variance by Components")
plt.xlabel("Number of Components")
plt.ylabel("Sum")

In [ ]:
th = 0.8
print("Con ", np.min(np.argwhere(pca.explained_variance_ratio_.cumsum() > th)), " Componentes, se supera el ", th, " de variabilidad.")

### Entrenar

In [ ]:
print('\n MLP:')
model_MLP = Pipeline([('SS',StandardScaler()),
                  ('PCA',PCA(n_components=50)),
                  ('MLP',MLPClassifier(hidden_layer_sizes=(100,100),solver='adam',
                                       activation='relu',batch_size=200,
                                       max_iter=2000,alpha=0.0001))])

model_MLP.fit(x_train,y_train.ravel())

print('Train score: ',model_MLP.score(x_train, y_train))
print('Test score: ',model_MLP.score(x_test, y_test))

In [ ]:
ypred = model_MLP.predict(x_test)

print('Metricas: \n', metrics.classification_report(y_test,ypred))
print('Confusion matrix: \n', metrics.confusion_matrix(y_test,ypred))

Solo si queremos guardar el modelo

In [ ]:
pickle.dump(model_MLP,open('MLP_MNIST.sav','wb'))

### Cargar modelo

Si en lugar de entrenar queremos utilizar un modelo previo

In [ ]:
model_MLP = pickle.load(open('MLP_MNIST.sav', 'rb'))

### Leer webcam y tomar foto

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

Detección del Aruco

El aruco nos va a servir para recortar la imagen y solo tomar el número para la predicción.

In [ ]:
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_100)
parameters =  cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(dictionary, parameters)

Predicción

In [ ]:
frame = cv2.imread("photo.jpg")
corners, ids, _ = detector.detectMarkers(frame)

if len(corners) > 0 and len(ids)==1:
    points = np.asarray(corners)
    points = points.reshape(4,2)
    points = points.astype(int)

    p3 = points[2,:]
    p4 = points[3,:]

    d = 2*(abs(p3[0]-p4[0]))

    # frame[y:y+h,x:x+w]
    number = frame[p3[1]:p3[1]+d,p3[0]:p3[0]+d]

    # rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 10)
    frame = cv2.rectangle(frame, (p3[0],p3[1]), (p3[0]+d,p3[1]+d), (0,255,0), 2)

    number = cv2.cvtColor(number,cv2.COLOR_BGR2GRAY)
    _ , number = cv2.threshold(number,100,255,cv2.THRESH_BINARY_INV)
    number = cv2.resize(number,(28,28))

    pattern = number.reshape(1,784)

    prediction = model_MLP.predict(pattern)
    print(prediction)

    number = cv2.resize(number,(100,100))
    plt.imshow(number, cmap="gray")